# Yahoo finance data download

# Configuration

Yahoo documentation: https://ranaroussi.github.io/yfinance/index.html
Repo: https://github.com/jrdi/yfinance

In [22]:
# !pip install yfinance requests beautifulsoup4 tqdm lxml

In [29]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import date
import warnings
import time

In [24]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [26]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies" # TODO monitor!

## Tickers download

In [27]:
try:
    response = requests.get(WIKI_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'lxml')

    sp500_table = soup.find('table', {'id': 'constituents'})

    if sp500_table:
        sp500_list_df = pd.read_html(str(sp500_table))[0]
        sp500_list_df.rename(columns={'Symbol': 'Ticker', 'GICS Sector': 'Sector'}, inplace=True)
        sp500_list_df = sp500_list_df[['Ticker', 'Security', 'Sector']]
        print(f"SP500 list downloaded: {len(sp500_list_df)} from wiki.")
        sp500_list_df['Ticker'] = sp500_list_df['Ticker'].str.replace('.', '-', regex=False)
    else:
        print("Table 'constituents' not found on Wikipedia page.")
        sp500_list_df = pd.DataFrame()

except requests.exceptions.RequestException as e:
    print(f"Error while downloading Wikipedia page: {e}")
    sp500_list_df = pd.DataFrame()
except Exception as e:
    print(f"An unexpected error occurred while processing the list of companies: {e}")
    sp500_list_df = pd.DataFrame()

SP500 list downloaded: 503 from wiki.


## Tickers marketcap

In [32]:
stock_data = []
if not sp500_list_df.empty:
    print("Downloading Market Cap information (may take a while)....")
    tickers_to_query = sp500_list_df['Ticker'].tolist()

    delay_between_requests = 10.0

    for ticker_symbol in tqdm(tickers_to_query, desc="Download info"):
        try:
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            market_cap = info.get('marketCap')
            sector = info.get('sector')

            if market_cap and sector:
                 stock_data.append({
                     'Ticker': ticker_symbol,
                     'Sector': sector,
                     'Market Cap': market_cap
                 })
            else:
               print(f"Missing data (Market Cap or Sector) for {ticker_symbol}")

            time.sleep(delay_between_requests)

        except Exception as e:
            print(f"Unable to download information for {ticker_symbol}: {e}")
            wiki_row = sp500_list_df[sp500_list_df['Ticker'] == ticker_symbol]
            if not wiki_row.empty:
                sector_from_wiki = wiki_row.iloc[0]['Sector']
                if sector_from_wiki:
                     stock_data.append({
                        'Ticker': ticker_symbol,
                        'Sector': sector_from_wiki,
                        'Market Cap': None
                    })
            time.sleep(delay_between_requests / 2)


    sp500_full_df = pd.DataFrame(stock_data)
    print(f"\nInformation was downloaded for {len(sp500_full_df)} companies.")

    sp500_full_df.dropna(subset=['Market Cap', 'Sector'], inplace=True)
    sp500_full_df['Market Cap'] = pd.to_numeric(sp500_full_df['Market Cap'])
    print(f"Number of companies after removing missing data: {len(sp500_full_df)}")
    print("\nSectors: ")
    print(sp500_full_df['Sector'].value_counts())

else:
    print("The list of S&P 500 companies is empty, skipping Market Cap downloads.")
    sp500_full_df = pd.DataFrame()

Download info:   0%|          | 0/503 [00:00<?, ?it/s]

429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MMM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MMM&crumb=Edge%3A+Too+Many+Requests


Unable to download information for MMM: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AOS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AOS&crumb=Edge%3A+Too+Many+Requests


Unable to download information for AOS: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABT&crumb=Edge%3A+Too+Many+Requests


Unable to download information for ABT: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABBV?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABBV&crumb=Edge%3A+Too+Many+Requests


Unable to download information for ABBV: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

## Stocks selecting

In [ ]:
selected_tickers = []
num_top_stocks = 10

if not sp500_full_df.empty:
    print(f"\nSelecting top {num_top_stocks} companies from each sector based on MarketCap...")
    grouped_by_sector = sp500_full_df.groupby('Sector')

    top_stocks_per_sector = grouped_by_sector.apply(
        lambda x: x.nlargest(num_top_stocks, 'Market Cap'),
        # include_groups=False # may be neccessary
    )

    if isinstance(top_stocks_per_sector.index, pd.MultiIndex):
         top_stocks_per_sector = top_stocks_per_sector.reset_index(level=0, drop=True)


    selected_tickers = top_stocks_per_sector['Ticker'].unique().tolist() # unique() na wszelki wypadek
    print(f"Selected {len(selected_tickers)} unique tickers from {len(sp500_full_df['Sector'].unique())} sectors.")
    # print("Selected tickers:", selected_tickers)
    # print(top_stocks_per_sector[['Sector', 'Market Cap']].head())
else:
    print("DataFrame sp500_full_df is null, companies cannot be downloaded.")

## Historical data downloading

In [ ]:
if selected_tickers:
    start_date = "2018-01-01"
    end_date = date.today().strftime('%Y-%m-%d')

    print(f"\nDownloading OHLCV data for {len(selected_tickers)} tickers from {start_date} to {end_date}...")
    try:
        all_ohlcv_data = yf.download(selected_tickers, start=start_date, end=end_date, progress=True)

        if all_ohlcv_data.empty:
            print("Failed to download any data OHLCV.")
        else:
            print("\nData download completed.")
            print("Shape of downloaded data: ", all_ohlcv_data.shape)
            print("\nExample :")
            print(all_ohlcv_data.head())

            # np. all_ohlcv_data.to_csv("sp500_top10_per_sector_ohlcv.csv")
            # print("\nData saved to: sp500_top10_per_sector_ohlcv.csv")


    except Exception as e:
        print(f"\nError occurred while downloading OHLCV data: {e}")

else:
    print("The list of selected tickers is empty. OHLCV data not downloaded.")

## Other approach

In [6]:
sp500_companies = pd.read_csv("sp500_companies/sp500_companies.csv")
sp500_companies.head(5)

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,254.49,3846819807232,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.069209
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,134.70,3298803056640,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.059350
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,436.60,3246068596736,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058401
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,224.92,2365033807872,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042550
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,191.41,2351625142272,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042309


In [5]:
sp500_companies.groupby("Sector").size()

Sector
Basic Materials           22
Communication Services    22
Consumer Cyclical         55
Consumer Defensive        37
Energy                    22
Financial Services        67
Healthcare                62
Industrials               70
Real Estate               31
Technology                82
Utilities                 32
dtype: int64

In [7]:
sp500_companies.groupby("Industry").size()

Industry
Advertising Agencies               2
Aerospace & Defense               12
Agricultural Inputs                4
Airlines                           3
Apparel Manufacturing              1
                                  ..
Utilities - Regulated Electric    23
Utilities - Regulated Gas          2
Utilities - Regulated Water        1
Utilities - Renewable              2
Waste Management                   2
Length: 114, dtype: int64

In [12]:
sp500_companies_sorted = sp500_companies.sort_values(by=['Sector', 'Marketcap'], ascending=[True, False])
sp500_top_10_per_sector = sp500_companies_sorted.groupby('Sector').head(10).reset_index(drop=True)

In [17]:
sp500_top_10_per_sector_tickers = sp500_top_10_per_sector['Symbol'].to_list()

In [19]:
TICKERS_TO_DOWNLOAD_FROM_YAHOO = sp500_top_10_per_sector_tickers
START_DATE="2010-01-01"
END_DATE="2025-01-01"
PERIOD = "1d"

# Data loading

In [20]:
data_list = {}
for ticker in TICKERS_TO_DOWNLOAD_FROM_YAHOO:
    try:
        data = yf.download(ticker, start=START_DATE, end=END_DATE, interval=PERIOD)
        data['ticker'] = ticker
        data_list[ticker] = data
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")

Failed to get ticker 'LIN' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['LIN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'SHW' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SHW']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'ECL' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ECL']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'APD' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['APD']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker 'FCX' r

KeyboardInterrupt: 

# Data show

In [8]:
data_list['SOL-USD'].columns

MultiIndex([('Adj Close', 'SOL-USD'),
            (    'Close', 'SOL-USD'),
            (     'High', 'SOL-USD'),
            (      'Low', 'SOL-USD'),
            (     'Open', 'SOL-USD'),
            (   'Volume', 'SOL-USD'),
            (   'ticker',        '')],
           names=['Price', 'Ticker'])

In [9]:
for ticker, df in data_list.items():
    print(f"**********{ticker}**********")
    # display(df)

**********USDT-USD**********
**********BTC-USD**********
**********XRP-USD**********
**********ETH-USD**********
**********BNB-USD**********
**********DOGE-USD**********
**********SOL-USD**********
**********STETH-USD**********
**********DOT-USD**********
**********TSLA**********
**********AAPL**********
**********NVDA**********
**********PLTR**********
**********SMCI**********
**********PFE**********
**********MARA**********
**********INTC**********
**********PLUG**********
**********F**********
**********AMZN**********
**********ZS**********
**********TSM**********
**********META**********
**********MRVL**********
**********NU**********


# Data save

In [10]:
# Save the data to a xlsx file
output_file = f"data/finance/popular_tickers/historical_data_{START_DATE}-{END_DATE}-{PERIOD}.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for ticker, df in data_list.items():
        df.to_excel(writer, sheet_name=ticker)

# Test

From: https://github.com/jrdi/yfinance

In [34]:
msft = yf.Ticker("MSFT")
msft.info

429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFT&crumb=Edge%3A+Too+Many+Requests


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
msft.actions

$MSFT: possibly delisted; no price data found  (1d 1926-04-23 -> 2025-03-29)


Series([], dtype: object)

In [12]:
msft.dividends

Date
2003-02-19 00:00:00-05:00    0.08
2003-10-15 00:00:00-04:00    0.16
2004-08-23 00:00:00-04:00    0.08
2004-11-15 00:00:00-05:00    3.08
2005-02-15 00:00:00-05:00    0.08
                             ... 
2023-11-15 00:00:00-05:00    0.75
2024-02-14 00:00:00-05:00    0.75
2024-05-15 00:00:00-04:00    0.75
2024-08-15 00:00:00-04:00    0.75
2024-11-21 00:00:00-05:00    0.83
Name: Dividends, Length: 84, dtype: float64

In [13]:
msft.splits

Date
1987-09-21 00:00:00-04:00    2.0
1990-04-16 00:00:00-04:00    2.0
1991-06-27 00:00:00-04:00    1.5
1992-06-15 00:00:00-04:00    1.5
1994-05-23 00:00:00-04:00    2.0
1996-12-09 00:00:00-05:00    2.0
1998-02-23 00:00:00-05:00    2.0
1999-03-29 00:00:00-05:00    2.0
2003-02-18 00:00:00-05:00    2.0
Name: Stock Splits, dtype: float64

In [15]:
msft.financials

,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Tax Effect Of Unusual Items,-99918000.0,-2850000.0,43754000.0,180160797.164637
Tax Rate For Calcs,0.182,0.19,0.131,0.138266
Normalized EBITDA,133558000000.0,105155000000.0,99905000000.0,83831000000.0
Total Unusual Items,-549000000.0,-15000000.0,334000000.0,1303000000.0
Total Unusual Items Excluding Goodwill,-549000000.0,-15000000.0,334000000.0,1303000000.0
Net Income From Continuing Operation Net Minority Interest,88136000000.0,72361000000.0,72738000000.0,61271000000.0
Reconciled Depreciation,22287000000.0,13861000000.0,14460000000.0,11686000000.0
Reconciled Cost Of Revenue,74114000000.0,65863000000.0,62650000000.0,52232000000.0
EBITDA,133009000000.0,105140000000.0,100239000000.0,85134000000.0
EBIT,110722000000.0,91279000000.0,85779000000.0,73448000000.0


In [16]:
msft.quarterly_financials

,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30
Tax Effect Of Unusual Items,57190000.0,-16263853.211009,-33660000.0,-29160000.0,-20700000.0
Tax Rate For Calcs,0.19,0.191339,0.18,0.18,0.18
Normalized EBITDA,37933000000.0,34416000000.0,33741000000.0,33556000000.0,31845000000.0
Total Unusual Items,301000000.0,-85000000.0,-187000000.0,-162000000.0,-115000000.0
Total Unusual Items Excluding Goodwill,301000000.0,-85000000.0,-187000000.0,-162000000.0,-115000000.0
Net Income From Continuing Operation Net Minority Interest,24667000000.0,22036000000.0,21939000000.0,21870000000.0,22291000000.0
Reconciled Depreciation,7383000000.0,6380000000.0,6027000000.0,5959000000.0,3921000000.0
Reconciled Cost Of Revenue,20099000000.0,19684000000.0,18505000000.0,19623000000.0,16302000000.0
EBITDA,38234000000.0,34331000000.0,33554000000.0,33394000000.0,31730000000.0
EBIT,30851000000.0,27951000000.0,27527000000.0,27435000000.0,27809000000.0


In [17]:
msft.institutional_holders

,Date Reported,Holder,pctHeld,Shares,Value
0,2023-06-30,Vanguard Group Inc,0.0879,653247546,222456919314
1,2023-06-30,Blackrock Inc.,0.0722,536245027,182612881494
2,2023-06-30,State Street Corporation,0.0395,293614078,99987338122
3,2023-06-30,"FMR, LLC",0.0283,210111250,71551285075
4,2023-06-30,Price (T.Rowe) Associates Inc,0.0215,159904219,54453782738
5,2023-06-30,"Geode Capital Management, LLC",0.0199,148129470,50444009713
6,2023-06-30,Morgan Stanley,0.0166,123498322,42056118573
7,2023-06-30,JP Morgan Chase & Company,0.0152,113024837,38489477991
8,2022-12-31,Norges Bank Investment Management,0.0116,86316926,20700525193
9,2023-06-30,Capital World Investors,0.0115,85507261,29118642660


In [27]:
msft.balance_sheet

,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Ordinary Shares Number,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Share Issued,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Net Debt,33315000000.0,12533000000.0,35850000000.0,43922000000.0
Total Debt,67127000000.0,59965000000.0,61270000000.0,67775000000.0
Tangible Book Value,121660000000.0,128971000000.0,87720000000.0,84477000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,75531000000.0,111256000000.0,104749000000.0,130256000000.0
Other Short Term Investments,57216000000.0,76552000000.0,90818000000.0,116032000000.0
Cash And Cash Equivalents,18315000000.0,34704000000.0,13931000000.0,14224000000.0
Cash Equivalents,6744000000.0,26226000000.0,5673000000.0,6952000000.0


In [18]:
msft.cashflow

,2024-06-30,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Free Cash Flow,74071000000.0,59475000000.0,65149000000.0,56118000000.0,NaN
Repurchase Of Capital Stock,-17254000000.0,-22245000000.0,-32696000000.0,-27385000000.0,NaN
Repayment Of Debt,-29070000000.0,-2750000000.0,-9023000000.0,-3750000000.0,NaN
Issuance Of Debt,29645000000.0,0.0,0.0,NaN,0.0
Issuance Of Capital Stock,2002000000.0,1866000000.0,1841000000.0,1693000000.0,NaN
Capital Expenditure,-44477000000.0,-28107000000.0,-23886000000.0,-20622000000.0,NaN
End Cash Position,18315000000.0,34704000000.0,13931000000.0,14224000000.0,NaN
Beginning Cash Position,34704000000.0,13931000000.0,14224000000.0,13576000000.0,NaN
Effect Of Exchange Rate Changes,-210000000.0,-194000000.0,-141000000.0,-29000000.0,NaN
Changes In Cash,-16179000000.0,20967000000.0,-152000000.0,677000000.0,NaN


In [26]:
msft.quarterly_cashflow

,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30
Free Cash Flow,19257000000.0,23322000000.0,20965000000.0,9118000000.0,20666000000.0
Repurchase Of Capital Stock,-4107000000.0,-4210000000.0,-4213000000.0,-4000000000.0,-4831000000.0
Repayment Of Debt,-966000000.0,-13065000000.0,-11589000000.0,-2916000000.0,-1500000000.0
Issuance Of Debt,0.0,5447000000.0,6352000000.0,10773000000.0,7073000000.0
Issuance Of Capital Stock,706000000.0,534000000.0,522000000.0,261000000.0,685000000.0
Capital Expenditure,-14923000000.0,-13873000000.0,-10952000000.0,-9735000000.0,-9917000000.0
End Cash Position,20840000000.0,18315000000.0,19634000000.0,17305000000.0,80452000000.0
Beginning Cash Position,18315000000.0,19634000000.0,17305000000.0,80452000000.0,34704000000.0
Effect Of Exchange Rate Changes,122000000.0,-103000000.0,-80000000.0,72000000.0,-99000000.0
Changes In Cash,2403000000.0,-1216000000.0,2409000000.0,-63219000000.0,45847000000.0


In [25]:
# show earnings
# msft.earnings
msft.income_stmt

,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Tax Effect Of Unusual Items,-99918000.0,-2850000.0,43754000.0,180160797.164637
Tax Rate For Calcs,0.182,0.19,0.131,0.138266
Normalized EBITDA,133558000000.0,105155000000.0,99905000000.0,83831000000.0
Total Unusual Items,-549000000.0,-15000000.0,334000000.0,1303000000.0
Total Unusual Items Excluding Goodwill,-549000000.0,-15000000.0,334000000.0,1303000000.0
Net Income From Continuing Operation Net Minority Interest,88136000000.0,72361000000.0,72738000000.0,61271000000.0
Reconciled Depreciation,22287000000.0,13861000000.0,14460000000.0,11686000000.0
Reconciled Cost Of Revenue,74114000000.0,65863000000.0,62650000000.0,52232000000.0
EBITDA,133009000000.0,105140000000.0,100239000000.0,85134000000.0
EBIT,110722000000.0,91279000000.0,85779000000.0,73448000000.0


In [23]:
# show next event (earnings, etc)
msft.calendar

{'Dividend Date': datetime.date(2024, 12, 12),
 'Ex-Dividend Date': datetime.date(2024, 11, 21),
 'Earnings Date': [datetime.date(2025, 1, 28), datetime.date(2025, 2, 3)],
 'Earnings High': 3.47,
 'Earnings Low': 3.08,
 'Earnings Average': 3.15862,
 'Revenue High': 70520000000,
 'Revenue Low': 68488700000,
 'Revenue Average': 68912341850}

In [22]:
# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

'-'

In [21]:
# show options expirations
msft.options

('2024-11-29',
 '2024-12-06',
 '2024-12-13',
 '2024-12-20',
 '2024-12-27',
 '2025-01-03',
 '2025-01-17',
 '2025-02-21',
 '2025-03-21',
 '2025-04-17',
 '2025-05-16',
 '2025-06-20',
 '2025-07-18',
 '2025-08-15',
 '2025-09-19',
 '2025-12-19',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18',
 '2027-01-15')